In [2]:
from functions import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [42]:
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/tests/area_1_parameters/sampling_shape/run_1762375871")
pcd = load_point_cloud(pcd_path)
# Load chen skeleton graph
chen_skeleton_graph = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")

visualize([VisObjGraph(chen_skeleton_graph), VisObjPCD(pcd)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Loading graph from ../output/tests/area_1_parameters/sampling_shape/run_1762375871/node_list.txt and ../output/tests/area_1_parameters/sampling_shape/run_1762375871/edge_list.txt...
Loaded graph with 2154 nodes and 2563 edges.


In [ ]:
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1.pcd")
chen_graph_path_input = Path("../output/tests/area_1_parameters/height/run_1762369371")
# output_path = Path("../MSSP/input/")
# mssp_output_path = Path("../MSSP/output/")

# chen_skeleton_ids = np.load(mssp_output_path / "chen_ske_ids.npy")
# chen_skeleton_dist = np.load(mssp_output_path / "chen_ske_dist.npy")

# Load point cloud
pcd = load_point_cloud(pcd_path)

# rasterize point cloud
voxel_grid, transformation_matrix = rasterize_point_cloud(pcd, voxel_size=0.1)

# fill holes
voxel_grid_flooded = flood_voxel_grid(voxel_grid, direction='up')
voxel_grid_flooded = flood_voxel_grid(voxel_grid_flooded, direction='down')
# invert it to get the free space
voxel_grid_inverted = invert_voxel_grid(voxel_grid_flooded)

# skeletonize
#skeleton = skeletonize_voxel_grid(voxel_grid_flooded, dilation_size=2)

# Load chen skeleton graph
chen_skeleton_graph = load_graph(chen_graph_path_input / "node_list.txt", chen_graph_path_input / "edge_list.txt")

visualize([VisObjGraph(chen_skeleton_graph), VisObjPCD(pcd)])
chen_skeleton_voxel, chen_skeleton_id_map = get_voxel_from_graph(chen_skeleton_graph, transformation_matrix, voxel_grid_flooded.shape)
visualize([VisObjVoxel(chen_skeleton_voxel, transformation_matrix), VisObjPCD(pcd)])

chen_visSkelDist = VisObjVoxel(chen_skeleton_dist, np.load(output_path / "M.npy"), noise=True)
visualize(chen_visSkelDist)

chen_visSkelIds = VisObjVoxel(chen_skeleton_ids, np.load(output_path / "M.npy"), noise=True)
visualize(chen_visSkelIds)

M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    if voxel_grid_inverted[tuple(v_pos_1.astype(int))] == 0:
        outlaiers.add(edge[0])
        print(f"Node {n1} maps to free space voxel {v_pos_1}")
    if voxel_grid_inverted[tuple(v_pos_2.astype(int))] == 0:
        outlaiers.add(edge[1])
        print(f"Node {n2} maps to free space voxel {v_pos_2}")

print(f"Total outlier nodes: {len(outlaiers)}")

visualize([VisObjGraph(chen_skeleton_graph, outlaiers), VisObjVoxel(voxel_grid_flooded, transformation_matrix), VisObjPCD(pcd)])

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1.pcd with 44026810 points.
Rasterizing point cloud with voxel size 0.1...
Voxel grid created with shape (248, 484, 54)
Filling voxel grid from direction up...
Voxel grid filled from direction up.
Filling voxel grid from direction down...
Voxel grid filled from direction down.
Inverting voxel grid...
Voxel grid inverted.
Loading graph from ../output/tests/area_1_parameters/height/run_1762278828/node_list.txt and ../output/tests/area_1_parameters/height/run_1762278828/edge_list.txt...
Loaded graph with 2873 nodes and 3346 edges.
Converting graph to voxel grid...
Graph converted to voxel grid.


NameError: name 'chen_skeleton_dist' is not defined

In [ ]:
(chen_skeleton_voxel & voxel_grid_flooded).sum()

np.int64(14)

In [13]:
from skimage.draw import line_nd
# inverse transformation matrix
M_inv = np.linalg.inv(transformation_matrix)

outlaiers = set()
for edge in chen_skeleton_graph.edges:
    n1 = chen_skeleton_graph.nodes[edge[0]]
    n2 = chen_skeleton_graph.nodes[edge[1]]
    coords_1 = n1['coords']
    coords_2 = n2['coords']
    coords_1_h = np.append(coords_1, 1)
    coords_2_h = np.append(coords_2, 1)
    v_pos_1 = (M_inv @ coords_1_h)[:3]
    v_pos_2 = (M_inv @ coords_2_h)[:3]
    # compute line voxels between v_pos_1 and v_pos_2
    line_voxels = np.array(line_nd(tuple(v_pos_1.astype(int)), tuple(v_pos_2.astype(int)))).T[1:] # skip the first entry, hence its the node itself
    for voxel in line_voxels:
        if voxel_grid[tuple(voxel)] == 1:
            outlaiers.add(edge)
print(f"Total outlier nodes: {len(outlaiers)}")

Total outlier nodes: 10


In [12]:
from skimage.draw import line_nd
for voxel in np.array(line_nd((0,0,0), (3,4,5))).T[1:]:
    print(voxel)

[1 1 1]
[1 2 2]
[2 2 3]
[2 3 4]


In [11]:
for i in range(5):
    for j in range(5):
        if i < 5-j:
            print(f"{i*0.4:.1f}, {3-j*0.4:.1f}")

0.0, 3.0
0.0, 2.6
0.0, 2.2
0.0, 1.8
0.0, 1.4
0.4, 3.0
0.4, 2.6
0.4, 2.2
0.4, 1.8
0.8, 3.0
0.8, 2.6
0.8, 2.2
1.2, 3.0
1.2, 2.6
1.6, 3.0


In [12]:
vals = [[1,2],[3,4],[5,6]]
# get all permutations of vals
import itertools
perms = list(itertools.permutations(vals))

In [13]:
perms

[([1, 2], [3, 4], [5, 6]),
 ([1, 2], [5, 6], [3, 4]),
 ([3, 4], [1, 2], [5, 6]),
 ([3, 4], [5, 6], [1, 2]),
 ([5, 6], [1, 2], [3, 4]),
 ([5, 6], [3, 4], [1, 2])]

In [ ]:
from functions import *
pcd_path = Path("../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd")
pcd = load_point_cloud(pcd_path)
bounding_box = np.array([[-26.0, -8.0, -1.0], [8.0, 50.0, 3.3]])
start_pt = np.array([-5.0, 0.0, 0.0])
visualize(VisObjPCD(pcd, bounding_box=bounding_box))

Loading point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd...
Loaded point cloud from ../modular_polygon_generation/libcore/data/maps/area_1_downsampled_0_1.pcd with 495103 points.
Adding bounding box to visualization...
Bounding box min: [-26.  -8.  -1.], max: [ 8.  50.   3.3]


In [2]:
bounding_box[0]

np.float64(-26.0)

In [ ]:
# independet variables
# min_wall_distance
# samping form
# non independent variables
# smpling_density - max_facets_grouped - frontier_split_threshold
# search_margin - resolution - voxel_size => resolution = search_margin * 2
# min_floor_height - max_floor_height

In [17]:
from pathlib import Path
import shutil
config_path = Path("../modular_polygon_generation/libcore/data/configs/tests/area_1_parameters/sampling_shape/")
copy_file = config_path / "copy.ini"

name_base = "area_1_shape_"

shapes = ["cube", "cuboid", "sphere", "spheroid", "cylinder"]
sampling_sizes = [25, 50, 100, 200, 400]

for shape in shapes:
    for size in sampling_sizes:
        # copy the copy file to a new file with the name format name_base + shape + "_" + str(size) + ".ini"
        new_file = config_path / f"{name_base}{shape}_{size:03d}.ini"
        # copy the file without changing the contents
        shutil.copy(copy_file, new_file)
